# Lý thuyết
Flower có sẵn rất nhiều strategy được đề xuất trong các bài báo như FedAvg, FedProx, ... Tuy nhiên Flower cũng cho phép tự tạo Strategy cho riêng mình.

Một Strategy để có thể chạy trên Flower cần phải có các hàm được viết trừu tượng sau:

In [ ]:
class Strategy(ABC):
    """Abstract base class for server strategy implementations."""

    @abstractmethod
    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize the (global) model parameters."""

    @abstractmethod
    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

    @abstractmethod
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate training results."""

    @abstractmethod
    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""

    @abstractmethod
    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation results."""

    @abstractmethod
    def evaluate(
        self, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate the current model parameters."""

Các hàm có mục đích như sau:
- initialize_parameters(): Khởi tạo mô hình ban đầu
- configure_fit(): Chọn client cũng như config cho mỗi round huấn luyện
- aggregate_fit(): tổng hợp các kết quả huấn luyện từ các client
- configure_evaluate() và aggregate_evaluate(): tương tự configure_fit() và aggregate_fit()
- evaluate(): hàm đánh giá mô hình từ phía server

Tuy nhiên thay vì viết lại từ đầu, tùy mục đích mà chúng ta muốn Strategy thực hiện mà chúng ta có thể viết một class kế thừa một Strategy có sẵn và viết đè lên các hàm cần thiết (thường là kế thừa FedAvg)

In [ ]:
class CustomStrategy(Strategy):
    # Nếu chúng ta chỉ cần sửa hàm aggregate_fit chỉ cần viết lại hàm này.
    def aggregate_fit(self, server_round, results, failures):
        # Your implementation here

# Ví dụ

Đây là Strategy FedCvM viết lại cách tổng hợp các mô hình

In [ ]:
from logging import WARNING
from typing import Callable, Dict, List, Optional, Tuple, Union
from functools import reduce

from flwr.common import (
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.fedavg import FedAvg
import numpy as np

class FedCvM(FedAvg):

    def aggregate_impurity(self, results: List[Tuple[NDArrays, int, float]]) -> NDArrays:
      sum_CvM = np.sum([CvM for _, _, CvM in results])
      weighted_weights = [
          [layer * CvM for layer in weights] for weights, _, CvM in results
      ]

      weights_prime: NDArrays = [
          reduce(np.add, layer_updates) / sum_CvM
          for layer_updates in zip(*weighted_weights)
      ]
      return weights_prime

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        if not results:
            return None, {}
        if not self.accept_failures and failures:
            return None, {}

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples, fit_res.metrics['CvM'])
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(
            self.aggregate_impurity(weights_results)
        )

        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)
        elif server_round == 1:  # Only log this warning once
            log(WARNING, "No fit_metrics_aggregation_fn provided")

        return parameters_aggregated, metrics_aggregated

Cần phải để ý là khi kết quả huấn luyện được gửi từ mô hình về Server có 3 giá trị là parameters - tham số của mô hình, num_examples - số lượng dữ liệu huấn luyện của client, metrics - các tham số custom khác

Xem lại hàm fit của class Client như sau:

In [ ]:
def fit(self, parameters, config):
        # Hàm huấn luyện
        self.model.set_weights(parameters)
        self.model.fit(self.trainset, epochs=1, verbose=VERBOSE)
        return self.model.get_weights(), len(self.trainset), {}

Ta thấy hàm này trả về 3 giá trị tương ứng với 3 giá trị mà hàm aggregate_fit sẽ nhận được. Trong đó '{}' là tương ứng với giá trị metrics. Giá trị metrics này là một dictionary để dễ nhận biết.